# Working with Data in Azure Machine Learning
This is a notebook for notes from the online modules from the DP-100 training in Azure. I think I did mod 2 in a myCIWorkstationRyan Jupyter Notebook which is why there's nothing here.

In [1]:
#sometimes I would wonder if I had managed to use the Python SDK but this proves it works on my computer, for a while I forget I had done the First Mod on this.
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

In [2]:
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

myCIWorkstationRyan : ComputeInstance
myCClusterRyan : AmlCompute


In [3]:
for ds_name in ws.datastores:
    print(ds_name)

blob_data
azureml_globaldatasets
workspacefilestore
workspaceblobstore


In [8]:
#register new blob datastore
blob_ds = Datastore.register_azure_blob_container(workspace = ws,
                                                 datastore_name='blob_data',
                                                 container_name='data_container',
                                                 account_name='az_store_acct',
                                                 account_key='12345678')

In [4]:
for ds_name in ws.datastores:
    print(ds_name)

blob_data
azureml_globaldatasets
workspacefilestore
workspaceblobstore


In [5]:
#gets the workspaces
workspace_blob_store = Datastore.get(ws, datastore_name='workspaceblobstore')
blob_store = Datastore.get(ws, datastore_name = 'blob_data')


In [7]:
#gets the default datastore
default_store = ws.get_default_datastore()
print(default_store)

{
  "name": "blob_data",
  "container_name": "data_container",
  "account_name": "az_store_acct",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [2]:
#sets a new default datastore - go to portal to view this change - does this change where the notebook goes to?
ws.set_default_datastore('workspaceblobstore')
type(ws)

NameError: name 'ws' is not defined

## Working directly with a datastore
You can work directly with a datastore to upload and download data by using the methods of the **datastore** class, like this:

In [1]:
blob_store.upload(src_dir = './digits-recognizer-kaggle/data',
                 target_path= 'digits-recognizer-kaggle/',
                 overwrite = True, show_progress=True)
#uploads a whole directory src_dr, could also use blob_store.upload_files(files = ['./digits-recognizer-kaggle/train.csv', 'digits-recognizer-kaggle/test.csv'], ...)
#this didn't work for me so I just decided to copy the learning information

blob_store.download(target_path='downloads',
                   prefix = '/data',
                    show_progress=True)

NameError: name 'blob_store' is not defined

## Using Data References
When you want to use a datasore in an experiment script, you must pass a data reference to the script. The data reference is configured for one of the following data access modes:
- **Download:** The contents of the path associated with teh data reference is downloaded toteh compute context where the experiment is running
- **Upload:** The files generated by your experiment script are uploaded to teh datastore after the run completes
- **Mount:** the path on the datastore is mounted as remote storage in the experiment compute context, enabling the contents to be accessed remotely (note that this mode is only available when teh experiment is run on a remote compute target - you cannot use this mode with local compute).

To pass the data reference to an experiment script, you must define a script parameter as shown here:

In [24]:
data_ref = blob_ds.path('data/files').as_download(path_on_compute='training_data')
estimator = SKLearn(source_directory='experiment_folder',
                    entry_script='training_script.py'
                    compute_target='local',
                    script_params = {'--data_folder': data_ref})

SyntaxError: invalid syntax (<ipython-input-24-53b53ffc8143>, line 4)

In your training script, you can retrieve the parameter and use it like a local folder:

In [3]:
import os
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--data_folder', type=str, dest='data_folder')
args = parser.parse_args()
data_files = os.listdir(args.data_folder)

usage: ipykernel_launcher.py [-h] [--data_folder DATA_FOLDER]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Ryan\AppData\Roaming\jupyter\runtime\kernel-8ed8c5db-0768-46bf-b147-49fe6370b133.json


SystemExit: 2

c:\users\ryan\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Introduction to Datasets
*Datasets* are versioned packaged data objects that can be easily consumed in experiments and pipelines. Datasets are the recommended way to work with data, and are the primary mechanism for advanced Azure Machine Learning capabilities like data labeling and data drift monitoring.
## Types of Datasets
Datasets are typically based on files in a datastore, though thaey can also be based on URLs and other sources. You can create the following types of dataset:
- **Tabular:** The data is read from the dataset as a table. You should use this type of dataset when your data is consistently structured and you want to work with it in common tabular data structures, such as Pandas dataframes.
- **Files:** The dataset presents a list of file paths that can be read as though from the file system. Use this type of dataset when your data is unstructured, or when you need to process the data at the file level (for example, to train a convolutional neural network from a set of image files).

# Creating and registering datasets
You can use the visual interface in Azure Machine Learning studio or the Azure Machine Learning SDK to create datasets from individual files or multiple file paths. The paths can include wildcards (for example, /files/*.csv) making it possible to encapsulate data from a large number of files in a single dataset.

After you've created a dataset, you can register it in the workspace to make it available for use in experiments and data processing pipelines later.

### Creating and Registering tabular datasets
To create a tabular dataset using the SDK, use ```from_delimited_files``` method of the ```Dataset.Tabular``` class, like this:

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/files/current_data.csv'),
             (blob_ds, 'data/files/archive/*.csv')]
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

The dataset in this example includes data from two file paths within the default datastore:
- the ```current_data.csv``` file in the ```data/files``` folder.
- all .csv files in the ```data/files/archive/``` folder.

After creating the dataset, the code registers it in the workspace with the name ```csv_table```.

### Creating and Registering file datasets
To create a file dataset using the SDK< use the ```from_files``` method of the ```Dataset.File``` class, like this:

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
file_ds = Dataset.File.from_files(path=(blob_ds, 'data/files/images/*.jpg'))
file_ds = file_ds.register(workspace=ws, name='img_files')

The dataset in the example includes all .jpg files in the ```data/files/images``` path within the default datastore:

After creating the dataset, the code registers it in the workspace with the name ```img_files```.

## Retrieving a registered dataset
After registering a dataset, you can retrieve it by using any of the following techniques:
- the ```datasets``` dictionary attribute of a ```Workspace``` object.
- the ```get_by_name``` or ```get_by_id``` method of the ```Dataset``` class.

Both of these techniques are shown in the following code;

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Get a dataset from the workspace datasets collection
ds1 = ws.datasets['csv_table']

# Get a dataset by name from the datasets class
ds2 = Dataset.get_by_name(ws, 'img_files')

## Datset versioning
Datasets can be versioned, enabling you to track historical versions of datasets that were used in experiments, and reproduce those experiments with data in the same state.

You can create a new version of a dataset by registering it with the same name as a previously registered dataset and specifying the ```create_new_version``` property:

In [ ]:
img_paths = [(blob_ds, 'data/files/images/*.jpg'),
             (blob_ds, 'data/files/images/*.png')]
file_ds = Dataset.File.from_files(path=img_paths)
file_ds = file_ds.register(workspace=ws, name='img_files', create_new_version=True)

In this example, the .png files in the ```images``` folder have been added to the definition of the ```img_paths``` dataset example used in the previous topic.

## Retrieving a specific dataset version
You can retrieve a specific version of a dataset by specifying the ```version``` parameter in the ```get_by_name``` method of the ```Dataset``` class.

In [ ]:
img_ds = Dataset.get_by_name(workspace=ws, name='img_files', version=2)

# Use datasets
You can read data directly from a dataset, or you can pass a dataset as a named *input* to a script configuration or estimator.

## Working with a dataset directly
If you have a reference to a dataset, you can access its contents directly.

For tabular datasets, most data processing begins by reading the dataset as a Pandas dataframe:

In [ ]:
df = tab_ds.to_pandas_dataframe()
# code to work with dataframe goes here

When working with a file dataset, you can use the ```to_path()``` method to return a list of the file paths encapsulated by the dataset:

In [ ]:
for file_path in file_ds.to_path():
    print(file_path)

## Passing a dataset to an experiment script
When you need to access a dataset in an experiment script, you can pass the dataset as an input to a ```ScriptRunConfig``` or an ```Estimator```. For example, the following code passes a tabular dataset to an estimator:

In [ ]:
estimator = SKLearn( source_directory='experiment_folder',
                     entry_script='training_script.py',
                     compute_target='local',
                     inputs=[tab_ds.as_named_input('csv_data')],
                     pip_packages=['azureml-dataprep[pandas]')

Since the script will need to work with a ```Dataset``` object, you must include either the full ```azureml-sdk``` package or the ```azureml-dataprep``` package with the ```pandas``` extra library in the script's compute environment.

In the experiment script itself, you can access the input and work with the ```Dataset``` object it references like this:

In [ ]:
run = Run.get_context()
data = run.input_datasets['csv_data'].to_pandas_dataframe()

When passing a file dataset, you must specify the access mode. For example:

In [ ]:
estimator = Estimator( source_directory='experiment_folder',
                     entry_script='training_script.py'
                     compute_target='local',
                     inputs=[img_ds.as_named_input('img_data').as_download(path_on_compute='data')],
                     pip_packages=['azureml-dataprep[pandas]')